In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import Dense, Dropout, Embedding, LSTM

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import os
import json

In [2]:
# 전처리한 학습 데이터를 불러옴
DATA_IN_PATH  = './data/'
DATA_OUT_PATH = './submission/'

TRAIN_INPUT_DATA = 'train_input.npy'
TRAIN_LABEL_DATA = 'train_label.npy'
DATA_CONFIGS = 'data_configs.json'

train_input = np.load(open(DATA_IN_PATH + TRAIN_INPUT_DATA, 'rb'))
train_label = np.load(open(DATA_IN_PATH + TRAIN_LABEL_DATA, 'rb'))
prepro_configs = json.load(open(DATA_IN_PATH + DATA_CONFIGS, 'r'))

In [3]:
# 랜덤 시드 고정
SEED_NUM = 1234
tf.random.set_seed(SEED_NUM)

In [4]:
# 모델 하이퍼파라미터 정의
model_name = 'rnn_classifier'
BATCH_SIZE = 128
NUM_EPOCHS = 5
VALID_SPLIT = 0.1
MAX_LEN = train_input.shape[1]

kargs = {'model_name': model_name,
        'vocab_size': prepro_configs['vocab_size'],
        'embedding_dimension': 100,
        'dropout_rate': 0.2,
        'lstm_dimension': 150,
        'dense_dimension': 150,
        'output_dimension': 1}

In [7]:
class RNNClassifier(tf.keras.Model):
    def __init__(self, **kargs):
        super(RNNClassifier, self).__init__(name=kargs['model_name'])
        self.embedding = Embedding(input_dim=kargs['vocab_size'],
                                   output_dim=kargs['embedding_dimension'])
        self.lstm_1_layer = LSTM(kargs['lstm_dimension'], return_sequences=True)
        self.lstm_2_layer = LSTM(kargs['lstm_dimension'])
        self.dropout = Dropout(kargs['dropout_rate'])
        self.fc1 = Dense(units=kargs['dense_dimension'], activation='tanh')
        self.fc2 = Dense(units=kargs['output_dimension'], activation='sigmoid')
    
    def call(self, x):
        x = self.embedding(x)
        x = self.dropout(x)
        x = self.lstm_1_layer(x)
        x = self.lstm_2_layer(x)
        x = self.dropout(x)
        x = self.fc1(x)
        x = self.dropout(x)
        x = self.fc2(x)
        
        return x

In [8]:
# 모델 생성
model = RNNClassifier(**kargs)
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [9]:
# overfitting을 막기 위한 ealrystop 추가
earlystop_callback = EarlyStopping(monitor='val_accuracy', min_delta=0.0001, patience=1)
# min_delta: the threshold that triggers the termination (acc should at least improve 0.0001)
# patience: no improvment epochs (patience = 1, 1번 이상 상승이 없으면 종료)

checkpoint_path = DATA_OUT_PATH + model_name + '/weights.h5'
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create path if exists
if os.path.exists(checkpoint_dir):
    print("{} -- Folder already exists \n".format(checkpoint_dir))
else:
    os.makedirs(checkpoint_dir, exist_ok=True)
    print("{} -- Folder create complete \n".format(checkpoint_dir))
    

cp_callback = ModelCheckpoint(
    checkpoint_path, monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=True)

./submission/rnn_classifier -- Folder already exists 



In [10]:
# 모델 학습
history = model.fit(train_input, train_label, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS,
                    validation_split=VALID_SPLIT, callbacks=[earlystop_callback, cp_callback])

Epoch 1/5
176/176 [==============================] - ETA: 0s - loss: 0.6924 - accuracy: 0.5169
Epoch 00001: val_accuracy improved from -inf to 0.54120, saving model to ./submission/rnn_classifier/weights.h5
176/176 [==============================] - 120s 680ms/step - loss: 0.6924 - accuracy: 0.5169 - val_loss: 0.6875 - val_accuracy: 0.5412
Epoch 2/5
176/176 [==============================] - ETA: 0s - loss: 0.6163 - accuracy: 0.6590
Epoch 00002: val_accuracy improved from 0.54120 to 0.78960, saving model to ./submission/rnn_classifier/weights.h5
176/176 [==============================] - 121s 685ms/step - loss: 0.6163 - accuracy: 0.6590 - val_loss: 0.5700 - val_accuracy: 0.7896
Epoch 3/5
176/176 [==============================] - ETA: 0s - loss: 0.5772 - accuracy: 0.7306
Epoch 00003: val_accuracy did not improve from 0.78960
176/176 [==============================] - 122s 691ms/step - loss: 0.5772 - accuracy: 0.7306 - val_loss: 0.6619 - val_accuracy: 0.5856


In [11]:
# val_accuracy did not improve from 0.78960, epochs=3